In [0]:
# Uncomment the line below and run this cell to get your data from github into colab (only runnable in colab, not ordinary jupyter notebook):
! git clone https://github.com/lkriener/music_generation.git && mv music_generation/* . && rm music_generation -r

Cloning into 'music_generation'...
remote: Enumerating objects: 495, done.
remote: Counting objects: 100% (495/495), done.
remote: Compressing objects: 100% (454/454), done.
remote: Total 495 (delta 64), reused 462 (delta 34), pack-reused 0
Receiving objects: 100% (495/495), 417.31 KiB | 8.35 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [0]:
# Uncomment line to install requirements
! pip install -r colab_requirements.txt

     |████████████████████████████████| 11.4MB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/ed/f6/fb/5d070524ecf7ba9ed201247a293c01945cfd7f840f8ef338c0
Successfully built pypianoroll


In [0]:
import src.midi_utils as midi_utils
import pygame
from pypianoroll import Multitrack, Track
import pypianoroll
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Test of pypianoroll library
We will first use the pypianoroll library - https://salu133445.github.io/pypianoroll/ - to convert a midi file into a multitrack object, get the soprano track, transpose it to C major, create a new multitrack object out of it, and finally write it to a new midi file. 

In [0]:
try:
    home_dir
except NameError:
    home_dir = os.getcwd()

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir

In [0]:
from src.dataset_utils import TrackDataset, get_dataset_representation_from_tracks
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
label_scaler = StandardScaler()

# iterate over all midi files of folder
import glob
import os
import numpy as np

try:
    home_dir
except NameError:
    home_dir = os.getcwd()
    

In [0]:
# determine boundaries in all tracks
lower_notes = []
upper_notes = []
list_pianorolls = []
os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
for midi_file in glob.glob("*.mid"):
    ## load midi file
    csv_text = midi_utils.load_to_csv(midi_file)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # convert to multitrack object
    multitrack = pypianoroll.parse(midi_file, beat_resolution=2)
    # get the soprano track object and transpose it to C major 
    track = pypianoroll.transpose(multitrack.tracks[0], -semitones)
    
    # get pitch range 
    pitch_range = track.get_active_pitch_range()
    lower_notes.append(pitch_range[0])
    upper_notes.append(pitch_range[1])
    
    # get the flattened representation of pianoroll
    pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
    # add it to the global list of all tracks
    list_pianorolls.append(pianoroll_flattened)

# convert into an array 
all_pianorolls = np.concatenate(list_pianorolls)

In [0]:
# get lower and upper bounds 
global_lower = min(lower_notes)
global_upper = max(upper_notes)
global_diff = global_upper - global_lower
n_notes = global_diff + 2 # we include the silence 
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 57
Global upper note : 83
Number of notes : 28


In [0]:
print(all_pianorolls[0:100])

[ 0  0 69  0 69  0 69 69 68 66 64 64 69 69 71 71 72  0 72 72 74 74 72 72
 71  0 71 71 72 72 74 74 76 74 72 72 77  0 77 77 76 74 76 76 74 74 74 74
 72  0 72 72 71 71 69 69 67 67 64 65 67  0 67 67 69 69 67 67 65 65 65 65
 64 64 76 76 74 74 72 72 71 71 71 71 69 69 69 69 69 69  0  0 68 68 70 70
 70 70 68 68]


In [0]:
# scale pianoroll to 0 
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print(all_pianorolls_scaled[0:100])

[ 0  0 13  0 13  0 13 13 12 10  8  8 13 13 15 15 16  0 16 16 18 18 16 16
 15  0 15 15 16 16 18 18 20 18 16 16 21  0 21 21 20 18 20 20 18 18 18 18
 16  0 16 16 15 15 13 13 11 11  8  9 11  0 11 11 13 13 11 11  9  9  9  9
  8  8 20 20 18 18 16 16 15 15 15 15 13 13 13 13 13 13  0  0 12 12 14 14
 14 14 12 12]


## Define a recurrent-network model 

In [0]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


## Define a LSTM network as for note-to-note melody generation
We now build a network to implement note-to-note melody generation using LSTMs units as well as drop-out of the output. This network is highly inspired from the Tutorial 08 of the class, by replacing characters by notes. 

In [0]:
# Declaring the model
class NoteRNN(nn.Module):
    
    def __init__(self, n_notes, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.n_notes = n_notes 
        #define the LSTM
        self.lstm = nn.LSTM(self.n_notes, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.n_notes)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Training Code

Time for training! We declare a function, where we define an optimizer (Adam) and loss (cross entropy). We then create the training and validation data and initialize the hidden state of the RNN. 
We loop over the training set, each time encoding the data into one-hot vectors, performing forward and backpropagation, and updating the network parameters.

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [0]:
# Declaring the train method
def train(net, data, data2=None, mode="melody_generation", epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: NoteRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    if mode == "harmonization":
        data2, val_data2 = data2[:val_idx], data2[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
        
    counter = 0
    n_notes = net.n_notes 
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        if mode == "melody_generation":
            batch_generator = midi_utils.get_pianoroll_batches(data, batch_size, seq_length)
        elif mode == "harmonization":
            batch_generator = midi_utils.get_pianoroll_batches_harmonization(data, data2, batch_size, seq_length)
        for x, y in batch_generator:
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = midi_utils.one_hot_encode_batch(x, n_notes)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                if mode == "melody_generation":
                    batch_generator_val = midi_utils.get_pianoroll_batches(val_data, batch_size, seq_length)
                elif mode == "harmonization":
                    batch_generator_val = midi_utils.get_pianoroll_batches_harmonization(val_data, val_data2, batch_size, seq_length)
                for x, y in batch_generator_val:
                    # One-hot encode our data and make them Torch tensors
                    x = midi_utils.one_hot_encode_batch(x, n_notes)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [0]:
# Define and print the net
n_hidden=256
n_layers=2

net = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 16
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, all_pianorolls_scaled, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [0]:
""# Defining a method to generate the next character
def predict(net, note, h=None):
        ''' Given a note, predict the next note.
            Returns the predicted note and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[note]])
        x = midi_utils.one_hot_encode_batch(x, net.n_notes)
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)
        

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        note_range = np.arange(net.n_notes)
        # select the likely next note with some element of randomness
        p = p.numpy().squeeze()
        note = np.random.choice(note_range, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return note, h

In [0]:
# Declaring a method to generate new melody
def sample(net, size, prime=[10,10,12,12]):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    notes = [no for no in prime]
    h = net.init_hidden(1)
    for no in prime:
        note, h = predict(net, no, h)
    notes.append(note)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        note, h = predict(net, notes[-1], h)
        notes.append(note)

    return np.array(notes)


# Declaring a method to generate new melody
def sample_harmonization(net, seq):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    size = len(seq)
    notes = []
    h = net.init_hidden(1)
    # Now pass in the previous character and get a new one
    for ii in range(size):
        note, h = predict(net, seq[ii], h)
        notes.append(note)

    return np.array(notes)




In [0]:
# Generating new melody
notes = sample(net, 100, prime=[1,1,1,1])
print("Generated sample :")
print(notes)
normal_pianoroll = midi_utils.unscale_pianoroll(notes, global_lower) # go back to the track range
print("Generated sample in the pitch range :")
print(normal_pianoroll)
# create a one_hot_pianoroll
one_hot_pianoroll = midi_utils.one_hot_encode_pianoroll(normal_pianoroll, 128)*90 # 90 for the velocity
# store it a in a track object
new_track = Track(pianoroll=one_hot_pianoroll, name='new track')
# create a multitrack made of the generated track object
new_multitrack = Multitrack(tracks=[new_track], tempo = 90, beat_resolution=2)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_track")


Generated sample :
[ 1  1  1  1 41 39 37 37  0 36 34 34 34 34  0 36 36 36 36 34 34 36 36 36
  0 39  0 39 39 39 37 39 41 41 41 41 41 39 39  0 39 34 34 34 36 37 37 38
 37 36 36 34 41 39  0 39 36 36 36 36 34 34 36 36 36 36  0 41 41 40 41 39
 37 37 37 37 36 36 34 34 34 36 32 34 34  0 36 36 36 36  0 34 36  0 36 36
 34 34 34 32 32 31 31  0 29]
Generated sample in the pitch range :
[35 35 35 35 75 73 71 71  0 70 68 68 68 68  0 70 70 70 70 68 68 70 70 70
  0 73  0 73 73 73 71 73 75 75 75 75 75 73 73  0 73 68 68 68 70 71 71 72
 71 70 70 68 75 73  0 73 70 70 70 70 68 68 70 70 70 70  0 75 75 74 75 73
 71 71 71 71 70 70 68 68 68 70 66 68 68  0 70 70 70 70  0 68 70  0 70 70
 68 68 68 66 66 65 65  0 63]


In [0]:
modified_midi_filename = home_dir + "/results/RNN_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

error: ignored

In [0]:
pygame.mixer.music.stop()

error: ignored

## Train for harmonization

In [0]:
# determine boundaries in all tracks
lower_notes = []
upper_notes = []
list_pianorolls = []
list_pianorolls2 = []
os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
for midi_file in glob.glob("*.mid"):
    ## load midi file
    csv_text = midi_utils.load_to_csv(midi_file)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # convert to multitrack object
    multitrack = pypianoroll.parse(midi_file, beat_resolution=2)
    # get the soprano track object and transpose it to C major 
    track = pypianoroll.transpose(multitrack.tracks[0], -semitones)
    track2 = pypianoroll.transpose(multitrack.tracks[3], -semitones)
    # get pitch range 
    pitch_range = track.get_active_pitch_range()
    lower_notes.append(pitch_range[0])
    upper_notes.append(pitch_range[1])
    pitch_range2 = track2.get_active_pitch_range()
    lower_notes.append(pitch_range2[0])
    upper_notes.append(pitch_range2[1])
    
    
    
    # get the flattened representation of pianoroll
    pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
    pianoroll_flattened2 = midi_utils.flatten_one_hot_pianoroll(track2.pianoroll)
    # add it to the global list of all tracks
    list_pianorolls.append(pianoroll_flattened)
    list_pianorolls2.append(pianoroll_flattened2)


In [0]:
# convert into an array 
all_pianorolls = np.concatenate(list_pianorolls)
all_pianorolls2 = np.concatenate(list_pianorolls2)
# get lower and upper bounds 
global_lower = min(lower_notes)
global_upper = max(upper_notes)
global_diff = global_upper - global_lower
n_notes = global_diff + 2 # we include the silence 
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 35
Global upper note : 83
Number of notes : 50


In [0]:
print("Voice to train on :")
print(all_pianorolls[0:100])
print("Voice to target :")
print(all_pianorolls2[0:100])

Voice to train on :
[ 0  0 69  0 69  0 69 69 68 66 64 64 69 69 71 71 72  0 72 72 74 74 72 72
 71  0 71 71 72 72 74 74 76 74 72 72 77  0 77 77 76 74 76 76 74 74 74 74
 72  0 72 72 71 71 69 69 67 67 64 65 67  0 67 67 69 69 67 67 65 65 65 65
 64 64 76 76 74 74 72 72 71 71 71 71 69 69 69 69 69 69  0  0 68 68 70 70
 70 70 68 68]
Voice to target :
[ 0  0 57 57 50 52 53 53 52  0 52 50 48 48 52 52 45 45 57 57 56 56 57 57
 52  0 52 52 57 57 55 55 48 48 53 55 57 57 59 59 60 59 57 55 53 53 55 55
 48 48 45 45 50 52 53 53 47 47 48 48 60 58 57 55 53 52 50 49 50 50 50 50
 44 44 45 45 47 47 48 50 52 52 40 40 45 45 45 45 45 45  0  0 56 56 54 54
 54 54 49 49]


In [0]:
# scale pianoroll to 0 
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print("Scaled voice to train on :")
print(all_pianorolls_scaled[0:100])
all_pianorolls_scaled2 = midi_utils.scale_pianoroll(all_pianorolls2, global_lower)
print("Scaled voice to target :")
print(all_pianorolls_scaled2[0:100])

Scaled voice to train on :
[ 0  0 35  0 35  0 35 35 34 32 30 30 35 35 37 37 38  0 38 38 40 40 38 38
 37  0 37 37 38 38 40 40 42 40 38 38 43  0 43 43 42 40 42 42 40 40 40 40
 38  0 38 38 37 37 35 35 33 33 30 31 33  0 33 33 35 35 33 33 31 31 31 31
 30 30 42 42 40 40 38 38 37 37 37 37 35 35 35 35 35 35  0  0 34 34 36 36
 36 36 34 34]
Scaled voice to target :
[ 0  0 23 23 16 18 19 19 18  0 18 16 14 14 18 18 11 11 23 23 22 22 23 23
 18  0 18 18 23 23 21 21 14 14 19 21 23 23 25 25 26 25 23 21 19 19 21 21
 14 14 11 11 16 18 19 19 13 13 14 14 26 24 23 21 19 18 16 15 16 16 16 16
 10 10 11 11 13 13 14 16 18 18  6  6 11 11 11 11 11 11  0  0 22 22 20 20
 20 20 15 15]


In [166]:
# Define and print the net
n_hidden=256
n_layers=2

net_harmonization = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 16
seq_length = 100
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net_harmonization, data=all_pianorolls_scaled, data2=all_pianorolls_scaled2, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

NoteRNN(
  (lstm): LSTM(50, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=50, bias=True)
)
[ 0  0 23 ... 14 14 21]
Epoch: 2/50... Step: 50... Loss: 2.8256... Val Loss: 2.9680
Epoch: 4/50... Step: 100... Loss: 2.4364... Val Loss: 2.6045
Epoch: 6/50... Step: 150... Loss: 1.4983... Val Loss: 1.6216
Epoch: 8/50... Step: 200... Loss: 0.4605... Val Loss: 0.5600
Epoch: 10/50... Step: 250... Loss: 0.1669... Val Loss: 0.2426
Epoch: 12/50... Step: 300... Loss: 0.0774... Val Loss: 0.1334
Epoch: 14/50... Step: 350... Loss: 0.0425... Val Loss: 0.0868
Epoch: 16/50... Step: 400... Loss: 0.0276... Val Loss: 0.0643
Epoch: 18/50... Step: 450... Loss: 0.0213... Val Loss: 0.0506
Epoch: 20/50... Step: 500... Loss: 0.0157... Val Loss: 0.0404


KeyboardInterrupt: ignored

In [0]:
# Generating harmonization

# get one voice from the dataset 
midi_filename = 'bwv14.5.mid'
csv_text = midi_utils.load_to_csv(midi_filename)
# get semitones to C major 
semitones,_ = midi_utils.get_semitones_to_C(csv_text)
print('Semitones to C major : ' +str(semitones))
# get multitrack object from midi 
multitrack = pypianoroll.parse(midi_filename, beat_resolution=2)
# get the soprano track
soprano_track = multitrack.tracks[0]
# transpose the track to C Major 
soprano_track = pypianoroll.transpose(soprano_track,-semitones)
soprano_pianoroll = midi_utils.flatten_one_hot_pianoroll(soprano_track.pianoroll)
print('Soprano pianoroll :')
print(soprano_pianoroll)
scaled_soprano_pianoroll = midi_utils.scale_pianoroll(soprano_pianoroll, global_lower)
# now we can predict the second voice 
scaled_alto_pianoroll = sample_harmonization(net_harmonization, scaled_soprano_pianoroll)
alto_pianoroll = midi_utils.unscale_pianoroll(scaled_alto_pianoroll, global_lower)
print('Alto pianoroll :')
print(alto_pianoroll)
one_hot_alto = midi_utils.one_hot_encode_pianoroll(alto_pianoroll, 128)*90
alto_track = Track(pianoroll=one_hot_alto, name='alto gernerated track')
# create a multitrack made of the initial soprano track and the generated alto
new_multitrack = Multitrack(tracks=[alto_track], tempo = 90, beat_resolution=2)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_harmonization_track")

Semitones to C major : -2
Soprano pianoroll :
[ 0  0 69 71 72 72 73 73 74 74 76 76 74 74 73 73 74  0 74 74 72 71 69 71
 72 74 76 76 74 72 74 74 72 72 69 71 72 72 73 73 74 74 76 76 74 74 73 73
 74  0 74 74 72 71 69 71 72 74 76 76 74 72 74 74 72 72 74 74 76 76 74 72
 71 71 72 72 74 72 71 71 69  0 69 69 74 74 73 73 74 74 76 74 72 71 69 69
 67  0 67 67 72 72 74 74 76 76 74 72 71 69 71 71 69 69]
Alto pianoroll :
[37  0 69 71 72 72 73 73 74 74 76 76 74 74 73 73 74  0 74 74 72 71 69 71
 72 74 76 76 74 72 74 74 72 72 69 71 72 72 73 73 74 74 76 76 74 74 73 73
 74  0 74 74 72 71 69 71 72 74 76 76 74 72 74 74 72 72 74 74 76 76 74 72
 71 71 72 72 74 72 71 71 69  0 69 69 74 74 73 73 74 74 76 74 72 71 69 69
 67  0 67 67 72 72 74 74 76 76 74 72 71 69 71 71 69 69]


In [0]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

error: ignored

In [0]:
pygame.mixer.music.stop()